In [1]:
import ctypes
import numpy as np
import timeit
import time
import knv as kNV
import matplotlib.pyplot as plt
import scipy

In [2]:
kokoDLL = ctypes.WinDLL("hello.dll")

In [3]:
lib = ctypes.cdll.LoadLibrary('hello.dll')
class GPURunner(object):
    def __init__(self):
        self.dim = 0
        lib.GPURunner_new.argtypes = []
        lib.GPURunner_new.restype = ctypes.c_void_p
        #lib.GPURunner_bar.argtypes = [ctypes.c_void_p]
        #lib.GPURunner_bar.restype = ctypes.c_void_p
        #lib.GPURunner_test.argtypes = [ctypes.c_void_p,np.ctypeslib.ndpointer(np.float32), np.ctypeslib.ndpointer(np.float32), ctypes.c_uint] 
        #lib.GPURunner_test.restype = ctypes.c_void_p
        #lib.GPURunner_propagate.argtypes = [ctypes.c_void_p, np.ctypeslib.ndpointer(np.complex64), np.ctypeslib.ndpointer(np.complex64), np.ctypeslib.ndpointer(np.complex64), np.ctypeslib.ndpointer(np.complex64), np.ctypeslib.ndpointer(np.float32), np.ctypeslib.ndpointer(np.float32), ctypes.c_uint, ctypes.c_uint] 
        #lib.GPURunner_propagate.restype = ctypes.c_void_p
        
        lib.GPURunner_readback.argtypes = [ctypes.c_void_p]
        lib.GPURunner_readback.restype = ctypes.c_void_p
        lib.GPURunner_sethamiltonian.argtypes = [ctypes.c_void_p, np.ctypeslib.ndpointer(np.complex64,flags='F'), np.ctypeslib.ndpointer(np.complex64,flags='F'), ctypes.c_uint] 
        lib.GPURunner_sethamiltonian.restype = ctypes.c_void_p
        
        lib.GPURunner_equiprop.argtypes = [ctypes.c_void_p, np.ctypeslib.ndpointer(np.complex64,flags='F'), ctypes.c_float, ctypes.c_uint, np.ctypeslib.ndpointer(np.complex64,flags='F')]
        lib.GPURunner_equiprop.restype = ctypes.c_void_p
        
        self.obj = lib.GPURunner_new()
        
    def __del__(self):
        lib.GPURunner_del.argtypes = [ctypes.c_void_p]
        lib.GPURunner_del.restype = ctypes.c_void_p
        lib.GPURunner_del(self.obj)
        
    
    def propagate(self,H0,H1,carr,dtarr):
        dim = np.shape(H0)
        dim = dim[0]
        pts = len(carr)
        #reH0 =np.float32(np.real(H0))
        #imH0 =np.float32(np.imag(H0))
        #reH1 =np.float32(np.real(H1))
        #imH1 =np.float32(np.imag(H1))
        #lib.GPURunner_propagate(self.obj,H0,H0,H1,H1,carr,dtarr,dim,pts)
    
    def set_hamiltonian(self,H0,H1):
        dim = np.shape(H0)
        dim = dim[0]
        self.dim = dim
        lib.GPURunner_sethamiltonian(self.obj,np.asfortranarray(H0),np.asfortranarray(H1),dim)
        
    def readback(self):
        lib.GPURunner_readback(self.obj)
    
    def equiprop(self, c_arr, dt):
        pts = len(c_arr)
        #print(np.asfortranarray(c_arr))
        output = np.empty(self.dim**2,dtype=np.complex64,order='F')
        lib.GPURunner_equiprop(self.obj,np.asfortranarray(c_arr),dt,pts,output)
        return np.reshape(output,(self.dim,self.dim))
        
    





In [18]:
neu = GPURunner()
id3 = np.identity(3,dtype=np.complex64)
id2 = np.identity(2,dtype=np.complex64)
H0 = np.complex64(np.array([[0,1],[1,0]]))
#H0 = np.kron(id3,H0)
#H0 = np.kron(id2,H0)
#H0 = np.kron(id2,H0)
#H0 = np.kron(id2,H0)


H1 = id2
#H0 = id2*1j
#carr = np.complex64(np.array([0,0,1,0,0,0,0,0,0]))
carr = np.complex64(np.zeros(80000))

dt = -np.pi/8
neu.set_hamiltonian(H0,H1)
#neu.readback()

In [19]:
np.shape(H0)

(2, 2)

In [20]:
carr = np.linspace(1,3,2,dtype=np.complex64)*0

In [21]:
#%%timeit
out_gpu = neu.equiprop(carr,dt)
kNV.npdisp(out_gpu)

<IPython.core.display.Latex object>

In [22]:
tmp = scipy.linalg.expm(-1j*dt*np.array([[0,1],[1,0]]))
tmp = tmp @ tmp
kNV.npdisp(tmp)

<IPython.core.display.Latex object>

In [9]:
k = 4
(-1j)**k*scipy.special.jv(k,2)

(0.03399571980756843+0j)

In [16]:
del neu

In [9]:
time_steps_CPU = range(100,40000,100)
store_CPU = []
for ts_once in kNV.bar(time_steps_CPU):
    id3 = np.identity(3,dtype=np.complex128)
    id2 = np.identity(2,dtype=np.complex128)
    H0 = np.complex64(np.array([[0,1],[1,0]]))
    H0 = np.kron(id3,H0)
    H0 = np.kron(id2,H0)
    #H0 = np.kron(id2,H0)
    H1 = H0

    tic = timeit.default_timer()
    for i in range(200):
        out_cpu = kNV.cos2pulse(H0,H1,1e9,0,1,1e-9,pts_per_cycle=ts_once)
    toc = timeit.default_timer()
    exec_time = (toc-tic)
    store_CPU.append(exec_time)
    
    
    

KeyboardInterrupt: 

In [ ]:
time_steps = range(100,15000,100)
store = []
for ts_once in kNV.bar(time_steps):
    neu = GPURunner()
    id3 = np.identity(3,dtype=np.complex64)
    id2 = np.identity(2,dtype=np.complex64)
    H0 = np.complex64(np.array([[0,1],[1,0]]))
    H0 = np.kron(id3,H0)
    H0 = np.kron(id2,H0)
    #H0 = np.kron(id2,H0)

    H1 = H0
    #carr = np.complex64(np.array([0,0,1,0,0,0,0,0,0]))
    carr = np.complex64(np.zeros(ts_once))

    dt = np.pi/4
    neu.set_hamiltonian(H0,H1)
    tic = timeit.default_timer()
    for i in range(200):
        out_gpu = neu.equiprop(carr,dt)
    toc = timeit.default_timer()
    exec_time = (toc-tic)
    del neu
    store.append(exec_time)

In [ ]:
gpuobj = GPURunner()
gpuobj.set_hamiltonian(H0,H1)
propagator = gpuobj.equiprop(c_arr,dt)

del gpuobj

In [ ]:
new_arr = []
for i in range(len(store)):
    new_arr.append(store[i]/time_steps[i])

In [ ]:
tmp = np.array(new_arr)*1e6
plt.plot(tmp[60:])


In [ ]:
new_arr_CPU = []
for i in range(len(store_CPU)):
    new_arr_CPU.append(store_CPU[i]/time_steps_CPU[i])

In [ ]:
tmp_CPU = np.array(new_arr_CPU)*1e6
plt.plot(tmp_CPU)

In [ ]:
%%timeit
out_cpu = kNV.cos2pulse(H0,H1,1e9,0,1,5000e-9,pts_per_cycle=10)

In [ ]:
del neu

In [ ]:
import scipy.linalg
out_scipy1 = scipy.linalg.expm(-1j*dt*(H0+0*H1))
out_scipy2 = scipy.linalg.expm(-1j*dt*(H0+1*H1))

out_scipy = out_scipy1@out_scipy2@out_scipy1
out_scipy

In [ ]:
out_scipy1@out_scipy2

In [ ]:
diff = np.reshape(out_gpu,(2,2)) - out_scipy

In [ ]:
diff

In [ ]:
print(np.finfo(np.float32).eps)

In [ ]:
1/2**23

In [ ]:
H0 = np.random.normal(size=(24,24)) + 1j*np.random.normal(size=(24,24)) 
H1 = np.random.normal(size=(24,24)) + 1j*np.random.normal(size=(24,24)) 
H0 = np.complex64(H0)
H1 = np.complex64(H1)
carr = np.float32(np.random.normal(size=80000))
tarr =np.float32(np.random.normal(size=80000))


In [ ]:
test = np.complex64(np.array([[1,4,7],[2,5,8],[3,6,9]]))

In [ ]:
neu.equiprop(test,3.0)

In [ ]:
neu.set_hamiltonian(test,test)

In [ ]:
neu.readback()

In [ ]:
del neu

In [ ]:
neu

In [ ]:
neu.propagate(test,test,carr,tarr)

In [ ]:
%%timeit
neu.propagate(H0,H1,carr,tarr)

In [ ]:
probes = (np.arange(40)+1)

In [ ]:
reps = 500

probes = (np.arange(40)+1)*10000
time_stor = []
for pi in kNV.bar(probes):
    H0 = np.random.normal(size=(24,24)) + 1j*np.random.normal(size=(24,24)) 
    H1 = np.random.normal(size=(24,24)) + 1j*np.random.normal(size=(24,24)) 
    carr = np.float32(np.random.normal(size=pi))
    tarr =np.float32(np.random.normal(size=pi))

    tic = time.time_ns()
    for i in range(reps):
        neu.propagate(H0,H1,carr,tarr)
    toc = time.time_ns()
    runtime = (toc-tic)/reps*1e-6
    time_stor.append(runtime)

    #print(runtime)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(probes/1e3,time_stor)

In [ ]:
transfer_per_sec = 4*10**9

In [ ]:
40*10000*32/8*2/transfer_per_sec*1e3

In [ ]:
testtimer = timeit.Timer()

In [ ]:
testtimer.

In [ ]:
%%timeit
neu.test(testarr)

In [ ]:
testarr

In [ ]:
#https://dbader.org/blog/python-ctypes-tutorial-part-2
func = kokoDLL.__getattr__("test_datatype")
func.restype = np.ctypeslib.ndpointer(np.complex64, ndim=1, flags='C')
#func.restype = np.ctypeslib.as_ctypes_type(np.float32)
#func.argtypes = [ctypes.c_int,ctypes.c_int]
#func.argname = ["A","B"]
#func.__doc__ = 'My docstring'

In [ ]:
func()

In [ ]:
neu = ctypes.CFUNCTYPE(None)

In [ ]:
def setupf():
    import ctypes
    import numpy as np
    import timeit
    kokoDLL = ctypes.WinDLL("hello.dll")
    lib = ctypes.cdll.LoadLibrary('hello.dll')
    class GPURunner(object):
        def __init__(self):
            lib.GPURunner_new.argtypes = []
            lib.GPURunner_new.restype = ctypes.c_void_p
            lib.GPURunner_bar.argtypes = [ctypes.c_void_p]
            lib.GPURunner_bar.restype = ctypes.c_void_p
            lib.GPURunner_test.argtypes = [ctypes.c_void_p,np.ctypeslib.ndpointer(np.float32), np.ctypeslib.ndpointer(np.float32), ctypes.c_uint] 
            lib.GPURunner_test.restype = ctypes.c_void_p
            lib.GPURunner_propagate.argtypes = [ctypes.c_void_p, np.ctypeslib.ndpointer(np.float32), np.ctypeslib.ndpointer(np.float32), np.ctypeslib.ndpointer(np.float32), np.ctypeslib.ndpointer(np.float32), np.ctypeslib.ndpointer(np.float32), np.ctypeslib.ndpointer(np.float32), ctypes.c_uint, ctypes.c_uint] 
            lib.GPURunner_propagate.restype = ctypes.c_void_p


            self.obj = lib.GPURunner_new()
        def bar(self):
            lib.GPURunner_bar(self.obj)
        def test(self,arr):
            dim = len(arr)
            rea = np.float32(np.real(arr))
            ima = np.float32(np.imag(arr))
            lib.GPURunner_test(self.obj,rea,ima,dim)
            return rea+1j*ima
        def propagate(self,H0,H1,carr,dtarr):
            dim = np.shape(H0)
            dim = dim[0]
            pts = len(carr)
            reH0 =np.float32(np.real(H0))
            imH0 =np.float32(np.imag(H0))
            reH1 =np.float32(np.real(H1))
            imH1 =np.float32(np.imag(H1))
            lib.GPURunner_propagate(self.obj,reH0,imH0,reH1,imH1,carr,dtarr,dim,pts)
    print("HI")       
    neu = GPURunner() 
    H0 = np.random.normal(size=(24,24)) + 1j*np.random.normal(size=(24,24)) 
    H1 = np.random.normal(size=(24,24)) + 1j*np.random.normal(size=(24,24)) 
    carr = np.float32(np.random.normal(size=80000))
    tarr =np.float32(np.random.normal(size=80000))
    #print(H0)

def run():
    print("HI2")
    print(H0)
    neu.propagate(H0,H1,carr,tarr)
    


In [ ]:
result = timeit.timeit(setup=setupf,stmt=run,number=100)

In [ ]:
result

In [ ]:
ApiProto = ctypes.WINFUNCTYPE(ctypes.c_void_p);

In [ ]:
test = ApiProto(("device_info",kokoDLL))

In [ ]:
test()

In [ ]:
ApiProto2= ctypes.WINFUNCTYPE(ctypes.c_void_p,ctypes.c_int,ctypes.c_int);

In [ ]:
adder = ApiProto2(("adder",kokoDLL))

In [ ]:
adder(1j,2)